# PU分類（奇数が正）のノートブック

In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
# ハイパーパラメータ
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2

In [3]:
# データセットの読み込み
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
# ラベルの書き換え（奇数が正）
X_train, y_train = rewrite_label_with_binary_setting(X_train, y_train)
X_val, y_val = rewrite_label_with_binary_setting(X_val, y_val)
X_test, y_test = rewrite_label_with_binary_setting(X_test, y_test)

In [5]:
# 1000個の正例を持つデータセットを作成する
X_train, y_train = rewrite_label_with_pu_setting(X_train, y_train, positive_size=1000)

In [6]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, init_score=np.full_like(y_train, np.log(1.), dtype=float), free_raw_data=False)
    train_data._pu_label = y_train
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "custom",
        "metric": "custom",
        "verbose": -1,
        # 学習が不安定なので、小さめの値を設定します
        # 今回は単純な検証目的なので、ご容赦ください
        "learning_rate": 0.01,
        "num_boost_round": 400,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        fobj=pu_loss_objective,
        feval=binary_metric,
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [7]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.7616666666666667
0.8043333333333333
0.8078333333333333
0.8018333333333333
0.8105833333333333
0.8116666666666666
0.81425
0.8131666666666667
0.816
0.8165
0.8214166666666667
0.82575
0.8271666666666667
0.82525
0.82975
0.8303333333333334
0.83025
0.8305
0.8295833333333333
0.8275
0.8281666666666667
0.8283333333333334
0.8285833333333333
0.8285
0.8306666666666667
0.83325
0.8335
0.8355
0.8365833333333333
0.8350833333333333
0.8375
0.8359166666666666
0.8368333333333333
0.8388333333333333
0.8384166666666667
0.8373333333333334
0.839
0.8383333333333334
0.83775
0.8369166666666666
0.8365
0.8369166666666666
0.8374166666666667
0.8365
0.8379166666666666
0.8361666666666666
0.8371666666666666
0.8369166666666666
0.8381666666666666
0.8365833333333333
0.8383333333333334
0.84
0.84
0.8399166666666666
0.8394166666666667
0.8395833333333333
0.8395833333333333
0.8405833333333333
0.8415833333333333
0.8425833333333334
0.8445
0.8438333333333333
0.84475
0.8458333333333333
0.8454166666666667
0.8465
0.8464166666666667
0

In [8]:
np.save("pu_valid_accuracies.npy", valid_accuracies)